6 + 39 = 45 --> "0639054"

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import random_neural_net_models.mingpt.adder as adder
import random_neural_net_models.mingpt.model as gpt_model
import random_neural_net_models.mingpt.trainer as gpt_trainer
import random_neural_net_models.mingpt.utils as gpt_utils
import random_neural_net_models.utils as utils

logger = utils.get_logger("nb")

In [ ]:
data_config = adder.DataConfig(ndigit=2)

In [ ]:
# construct train and test datasets
train_dataset = adder.AdditionDataset(data_config, split=gpt_utils.SETS.train)
test_dataset = adder.AdditionDataset(data_config, split=gpt_utils.SETS.test)

In [ ]:
# get default config and overrides from the command line, if any
config = adder.get_config(
    vocab_size=train_dataset.get_vocab_size(),
    block_size=train_dataset.get_block_size(),
    max_iters=100,
)

print(config)

In [ ]:
# gpt_utils.setup_logging(config)
gpt_utils.set_seed(config.system.seed)

In [ ]:
model = gpt_model.GPT(config.model)

In [ ]:
# construct the trainer object
trainer = gpt_trainer.Trainer(config.trainer, model, train_dataset)

In [ ]:
# iteration callback


def batch_end_callback(trainer: gpt_trainer.Trainer):
    if trainer.iter_num % 10 == 0:
        logger.info(
            f"iter_dt {trainer.iter_dt * 1000:.2f} ms; iter {trainer.iter_num:_d}: train loss {trainer.loss.item():.5f}"
        )

    model.train()


trainer.set_callback("on_batch_end", batch_end_callback)

In [ ]:
# run the optimization
trainer.run()

In [ ]:
for x, y in test_dataset:
    logger.info(f"x: {x}")
    logger.info(f"y: {y}")
    pred = model.generate(x.unsqueeze(0), 1, do_sample=False)
    logger.info(f"pred: {pred}")
    break